### Reprojecting Precinct-Level Voting Results to Census Block Groups

This script mapped the 2016 election voting results in Texas from precincts to census block group. 

The precinct-level voting results are made publicly available by the *Voting and Election Science Team* at University of Florida and Wichita State University. The data can be downloaded from: https://dataverse.harvard.edu/dataset.xhtml?persistentId=doi:10.7910/DVN/NH5S2I

**Input datasets**:
   
- `Census blocks shapefile for Texas`: Shapefiles by states that define the shape of blocks. The Dataset can be downloaded by states from https://www.census.gov/geographies/mapping-files/time-series/geo/tiger-line-file.html

(The dataset is renamed after the download)

- `Census block groups shapefile for Texas`: Shapefiles by states that define the shape of block groups. Dataset can be downloaded by states from https://www.census.gov/geographies/mapping-files/time-series/geo/tiger-line-file.html

(The dataset is renamed after the download)

- `Precinct shapefile for Texas`: Shapefiles by states that define the shape of precincts. Dataset can be downloaded by states from https://dataverse.harvard.edu/dataset.xhtml?persistentId=doi:10.7910/DVN/NH5S2I 

(The dataset is renamed after the download)


**Output datasets**: 
    
- `texasBlkSize.xlsx`: Sizes of Texas blocks obtained from Census Bureau API (https://api.census.gov/data/2010/dec/sf1/variables.html). There is no single dataset that contain the size information for all Texas blocks. The downloadable datasets are organized by counties. 

- `Texas_bg_voting.`: Voting results from precinct mapped to block group

In [1]:
import geopy
from geopy.geocoders import Nominatim

import shapefile

from shapely.geometry import Point, Polygon
from shapely.geometry import shape
from shapely.ops import cascaded_union

import geopandas as gpd


import matplotlib.pyplot as plt

import rasterio
from rasterio.plot import show

import time

import pandas as pd
import numpy as np

import requests
import json

import xml.etree.ElementTree as ET #parsing xml response from API

import glob, os, pyproj
import copy

import urllib.request

import wget

from io import StringIO

#### Preparation: Obtaining sizes of blocks

On the Census Bureau website, dataset about blocks can only be downloaded by counties. Texas contains 254 counties, so it would be inefficient to download these data through the interface. Fortunately, queries can be made through the `census API.`

The variable `mykey` below is the key to access the API. Request the key through https://api.census.gov/data/key_signup.html to run the web-request codes.

I also saved Texas block sizes in an excel file. To move on to the next step, run the last line to read `texasBlkSize.xlsx`. 

In [ ]:
texasCtyTra= texasBlk.loc[:, ["COUNTYFP10", "TRACTCE10"]].drop_duplicates()

In [ ]:
blkSizeDf= pd.DataFrame(columns= ["P001001","NAME", "state", "county", "tract", "block"])
for row in texasCtyTra.itertuples():
    county= getattr(row, "COUNTYFP10")
    tract= getattr(row, "TRACTCE10")
    
    # variable `P001001` refers to total population
    
    myKey= "?????" #Census API key
    
    url= "https://api.census.gov/data/2010/dec/sf1?"+\
    "get=P001001,"+\
    "NAME&for=block:*"+\
    "&in=state:{}".format("48")+\
    "%20county:{}%20tract:{}".format(county, tract)+\
    "&key="+ myKey
    
    r= requests.get(url)
    c= str(r.content, "utf-8")
    c= c[1:-1].replace(",\n", "\n").replace("[", "").replace("]", "")
    
    c= StringIO(c)
    tractDf= pd.read_csv(c, sep= ",")
    
    blkSizeDf= blkSizeDf.append(tractDf, ignore_index= True)

blkSizeDf.to_excel("texasBlkSize.xlsx")

In [2]:
blkSizeDf= pd.read_excel("Data_TexasExample/texasBlkSize.xlsx")

#### Reprojecting the data

Read Texas block, block group and precinct boundaries

In [4]:
#Read Texas blocks shapefile. I renamed the file after the download
shp= glob.glob("Data_TexasExample/Data_censusBlocks/tl_2019_tabblock10_Texas/*.shp")[0]
texasBlk= gpd.read_file(shp) 
print(texasBlk.crs)

#Read Texas block group shapefile. I rename the file after the download
shp= glob.glob("Data_TexasExample/Data_censusBG/tl_2019_Texas_bg/*.shp")[0] 
texasBg= gpd.read_file(shp)
print(texasBg.crs)

#Read Texas precinct shapefiles. I rename the file after the download 
texasPct= gpd.read_file("Data_TexasExample/Data_votingResults/"+"Texas"+"_2016/"+"TX"+"_2016.shp") 
texasPct_reProj= texasPct.to_crs({'init' :'epsg:4269'}) #Reproject precinct map's crs
print(texasPct_reProj.crs)

{'init': 'epsg:4269'}
{'init': 'epsg:4269'}


/anaconda3/lib/python3.6/site-packages/pyproj/crs.py:77: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method.
  return _prepare_from_string(" ".join(pjargs))


{'init': 'epsg:4269'}


**Are precincts consist of blocks? **

The precinct-level voting results are mapped to block groups through blocks. If both precincts and block groups consist of blocks, the mapping is more straightforward.


In [3]:
#Block boundary overlapping with precinct boundary is a problem
#I trim the block boundaries a little bit to establish the `contains` relationship
texasBlk_trim= copy.copy(texasBlk)
texasBlk_trim["geometry"]= texasBlk_trim["geometry"].buffer(distance=-0.0000001)

In [4]:
#The `sjoin` function is the key step. 
#For every row in left dataset, the function compares with all rows in the right dataset.
#The operation is split into two steps because the dataset is big.
#The code outputs a dataset `merge` in which each row is an intersection between a block and precinct(s)
    #If a block is contatined in a precinct, the block corresponds to one row in the output dataset
    #If a block spreads acrossed x precincts, the block corresponds to x rows in the output dataset
    
merge1= gpd.sjoin(texasBlk_trim.loc[0:500000, :], texasPct_reProj, \
                  how= "left", op= "intersects")
merge2= gpd.sjoin(texasBlk_trim.loc[500001:, :], texasPct_reProj, \
                  how= "left", op= "intersects")

merge= merge1.append(merge2, ignore_index= True)

#Note that because texasBlk_trim is one the left, geometry of the output dataset `merge` are about blocks

In [5]:
#dictionary keys are block ID, dictionary values are frequency
countDic= {i:0 for i in merge.GEOID10}
for geoid in merge.GEOID10:
    countDic[geoid]+= 1

blk1= []
blk2up= []
for key, value in countDic.items():
    if value== 1:
        blk1+= [key]
    else:
        blk2up+= [key]

print(len(blk1)) #How many block is uniquely contained in one precinct
print(len(blk2up)) #How many blocks spread across 2 and more precincts

merge_blkMulti= merge.loc[merge.GEOID10.isin(blk2up), :]
pct2up= list(set(merge_blkMulti.PCTKEY)) #Precinct corresponding to the 
                                         #blocks that spread acorss multiple precincts

873323
40908


For (the small amount of) blocks that srpead across more than one precincts... How are their areas distributed across the multiple precincts?

In [7]:
texasBlk_multi= texasBlk_trim.loc[texasBlk_trim.GEOID10.isin(blk2up), :]
texasPct_multi= texasPct_reProj.loc[texasPct_reProj.PCTKEY.isin(pct2up), :]

In [8]:
#Form union geometries between the non-unique blocks and the corresponding precincts 
    #The unions that has block membership (i.e., has GEOID10) should be the 
    #same as the sjoin intersections above stored in the `merge` dataset. 
BlkPct_multi= gpd.overlay(texasBlk_multi, texasPct_multi, how= "union")

In [9]:
BlkPct_multi["intersectArea"]= BlkPct_multi.area #Note that these intersect areas are 
                                                 #slightly smaller because of trimming
    
    #Theoretically, the sum of the intersect areas within a block should
    #equal to the block area. But because I tirmmed the areas by 0.0000001
    #the sum of intersect areas will be slightly smaller than the block area
    #The difference is very small, but I do not want the small differences to 
    #aggregrate. Therefore, when I compute the percentages of a block's area that  
    #belong to different precincts (`blkInPct`), I estimate the block's area from 
    #its intersect geometries. This makes sure that the intersect areas sum up
    #to 100%. For example, for block 1, 20% of its area is in precinct A, 50% 
    #in precinct B, and 30% in precinct C. 

toMerge= pd.DataFrame(BlkPct_multi.groupby("GEOID10")["intersectArea"].sum())
toMerge.columns= ["blkArea_est"]

BlkPct_multi= pd.merge(BlkPct_multi, toMerge, on= "GEOID10")


#merge the intersects' area back to the above dataset created by `sjoin`
toMerge2= BlkPct_multi.loc[pd.notnull(BlkPct_multi.GEOID10), 
                          ["GEOID10", "PCTKEY", "intersectArea", "blkArea_est"]]

merge_blkMulti= pd.merge(merge_blkMulti, toMerge2, 
                         left_on= ["GEOID10", "PCTKEY"], 
                         right_on= ["GEOID10", "PCTKEY"])

#`blkInPct`= Area of intersect / Area of block: 
#this is the % of block that lies in different precincts
merge_blkMulti["blkInPct"]= merge_blkMulti["intersectArea"]/merge_blkMulti["blkArea_est"]

In [10]:
print(sum(merge_blkMulti.blkInPct>0.99), sum(merge_blkMulti.blkInPct>0.99)/len(blk2up))

28359 0.6932384863596363


For a substantial number of blocks that seemingly spread across multiple precincts, 99% of these block areas are actually within one precinct. This seems more like a boundary misalign, rather than the block truely spreading across multiple precincts. 

For the blocks that 99% of their area are within one precinct, there might not be a need to further split it, so I assign the block to that precinct.

In [ ]:
blkKindof1= list(merge_blkMulti.loc[merge_blkMulti.blkInPct>0.99, "GEOID10"])
blk1New= blk1+ blkKindof1
blk2upNew= list(set(blk2up)-set(blkKindof1))

In [ ]:
merge_blkMulti_judge= merge_blkMulti.loc[~merge_blkMulti.GEOID10.isin(blkKindof1), :]

In [ ]:
merge_blkMulti_judge.shape

To sum the block of codes above

Texas contains 914231 blocks. I trim the block boundaries by - 0.0000001 (around 10mm). 873323 (95.5%) of them are uniquely contained within one precinct. The remaining 40908 spread across more than one precinct. For these 40908 blocks, 28359 (69%) of them have the majority of their area (more than 99% of area) within one precinct.

Generally, it is safe to conclude that precincts are consisted of blocks. 

Here I construct a new/updated version of the `merge` dataset by combining the below two scenarios:

- For the 873323 + 28359 blocks that are uniquely contained within one precinct (either exactly or has 99% of its area in one precinct), the block `intersectArea` with precinct is the same as the block area, and the block `blkInPct` percentage is 1. Again, each of these block correspond to 1 row in the `merge` dataset. 

- For the 12549 blocks that spread across more than one precinct, their `intersectArea` and `blkInPct` percentages are computed in in the `merge_blkMulti` dataset. Again, if a block spread across x precincts, they correspond to x rows.


In [ ]:
#For blocks that are truely uniquely contained in one precinct:
merge_blkUnique1= copy.copy(merge.loc[merge.GEOID10.isin(blk1), :]) 

#For blocks that each ha 99% of its area in one precinct and thus determined to be in one precinct:
merge_blkUnique2= merge_blkMulti.loc[merge_blkMulti.GEOID10.isin(blkKindof1), :]
merge_blkUnique2= \
    merge_blkUnique2.sort_values(["GEOID10", "blkInPct"]).drop_duplicates("GEOID10", keep= "last")

merge_blkUnique_judge= \
    merge_blkUnique1.append(merge_blkUnique2.drop(columns= ["blkInPct"]), sort= True)

merge_blkUnique_judge["blkArea_est"]= merge_blkUnique_judge.area
merge_blkUnique_judge["intersectArea"]= merge_blkUnique_judge.area
merge_blkUnique_judge["blkInPct"]= 1

In [ ]:
merge_new= merge_blkUnique_judge.append(merge_blkMulti_judge, sort= True)
merge_new= merge_new.reset_index()

I now join the block size information stored in `blkSizeDf` to the `merge_new` dataset.

In [ ]:
#Creating IDs for joining datasets
merge_new["stateId"]= pd.to_numeric(merge_new["GEOID10"].str[:2])
merge_new["countyId"]= pd.to_numeric(merge_new["GEOID10"].str[2:5])
merge_new["tractId"]= pd.to_numeric(merge_new["GEOID10"].str[5:11])
merge_new["blockId"]= pd.to_numeric(merge_new["GEOID10"].str[11:15])

blkPct_intersectDF= pd.merge(merge_new, blkSizeDf, 
         left_on= ["stateId", "countyId", "tractId", "blockId"],
         right_on= ["state", "county", "tract", "block"])

In [ ]:
#Notice that there are acltually two blocks that are not in any precinct. 
#It's not uncommon for no-people blocks to be not included in any precinct: 

print(sum(pd.isnull(blkPct_intersectDF.PCTKEY)))
#blkPct_intersectDF.loc[pd.isnull(blkPct_intersectDF.PCTKEY), :]

In [ ]:
# population size of an intersect = 
# population size of its parent block * % the intersect area accounting for the block area

#The assumption is that the population is evenly distributed within a block

blkPct_intersectDF["intersectPopSize"]= \
    blkPct_intersectDF["blkInPct"]*blkPct_intersectDF["P001001"]

In [ ]:
#Population sizes of precincts are obtained through aggregrating population sizes 
#of blocks and intersects within them
helper= blkPct_intersectDF.groupby("PCTKEY")["intersectPopSize"].sum()
helper= pd.DataFrame(helper)
helper.columns= ["pctPopSize"]


blkPct_intersectDF= pd.merge(blkPct_intersectDF, helper, 
         left_on= ["PCTKEY"], right_on= ["PCTKEY"])

In [ ]:
#Precinct population sizes are estimated through summing up population sizes
#of smaller areas within them; the population sizes of the smaller areas are obtained
#through census. HOWEVER, some preincts are sized 0 based on the calculation. 
#It could be becuase of changes in population distribution. After all, the election
#was in 2016 and the census was in 2010. 
#But these precincts are all weirdly small in size (e.g., 50 times smaller than a block)

#Eitherway, voting results in these precincts cannot be mapped to block groups. 

weirdPct= set(blkPct_intersectDF.loc[blkPct_intersectDF["pctPopSize"]== 0, ].PCTKEY)
print("# of weird precincts:", len(weirdPct))

print("# of people in these weird precincts who voted for Trump:",
     texasPct_reProj.loc[texasPct_reProj.PCTKEY.isin(weirdPct), :][["G16PRERTru"]].sum())
print("# of people in these weird precincts who voted for Cliton:",
     texasPct_reProj.loc[texasPct_reProj.PCTKEY.isin(weirdPct), :][["G16PREDCli"]].sum())


print("# of corresponding blocks:",
      blkPct_intersectDF.loc[blkPct_intersectDF.PCTKEY.isin(weirdPct), "P001001"].shape)
print("# of people in these blocks:",
      blkPct_intersectDF.loc[blkPct_intersectDF.PCTKEY.isin(weirdPct), "P001001"].sum())


#There are 1923 blocks corresponding to these problematic precincts. 
#Base on census block enumeration, there would be 0 people in these precincts;
#but the number of people who voted in these precincts is not zero 
#(578 for Trump and 675 for Cliton).

#As the result, the total number of the votes mapped to block groups would
#be smaller than the number of votes recorded in the original precinct dataset

In [ ]:
#I assign 1 people in these werid precincts to avoid dividing by zero.
blkPct_intersectDF.loc[blkPct_intersectDF.PCTKEY.isin(weirdPct), "pctPopSize"]= 1

#Voting results of the blocks/intersects. If a block accounts for 5% of the 
#population of the precincts, its voting results would be 5% times the numbers of 
#people voting for different candidates

blkPct_intersectDF["intersectTru"]= \
        blkPct_intersectDF["intersectPopSize"]/blkPct_intersectDF["pctPopSize"]*\
        blkPct_intersectDF["G16PRERTru"]
        
blkPct_intersectDF["intersectCli"]= \
        blkPct_intersectDF["intersectPopSize"]/blkPct_intersectDF["pctPopSize"]*\
        blkPct_intersectDF["G16PREDCli"]

blkPct_intersectDF["intersectJoh"]= \
        blkPct_intersectDF["intersectPopSize"]/blkPct_intersectDF["pctPopSize"]*\
        blkPct_intersectDF["G16PRELJoh"]

blkPct_intersectDF["intersectSte"]= \
        blkPct_intersectDF["intersectPopSize"]/blkPct_intersectDF["pctPopSize"]*\
        blkPct_intersectDF["G16PREGSte"]

blkPct_intersectDF["intersectOth"]= \
        blkPct_intersectDF["intersectPopSize"]/blkPct_intersectDF["pctPopSize"]*\
        blkPct_intersectDF["G16PREOth"]

Now I aggregrate the intersect/block level data to the block level to estimate the voting results in blocks.

In [ ]:
blkVote= blkPct_intersectDF.groupby("GEOID10")[["intersectTru", "intersectCli",
                                               "intersectJoh", "intersectSte",
                                               "intersectOth"]].sum()
blkVote.columns= ["blkTru", "blkCli", "blkJoh", "blkSte", "blkOth"]

Now I can aggregrate blocks to block groups and get block group voting results. I then merge the block group level voting results back to the original block group shapefiles. This completes the mapping operation.

In [ ]:
blkVote["GEOID10"]= blkVote.index
blkVote["bgID"]= blkVote["GEOID10"].str[:-3]

In [ ]:
bgVote= blkVote.groupby("bgID")[["blkTru", "blkCli", "blkJoh", "blkSte", "blkOth"]].sum()
bgVote.columns= ["bgTru", "bgCli", "bgJoh", "bgSte", "bgOth"]

In [ ]:
bgVote["GEOID"]= bgVote.index

In [ ]:
texasBg_wVote= pd.merge(texasBg, bgVote, on= "GEOID")

In [ ]:
outfp= "texasBg_estimateVoting/texasBg_estimateVoting.shp"
texasBg_wVote.to_file(outfp)